In [1]:
import torch

# Neural Network Basic Component Implementations

## Accuracy

In [2]:
class Accuracy:
    def __init__(self):
        self.output = 0
    def forward(self,y_pred,y_true):
        if y_pred.shape != y_true.shape:
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)),y_true] = 1
        else:
            one_hot_notation = y_true
        correct_values = y_pred==one_hot_notation
        correct_values = correct_values * one_hot_notation
        self.output = torch.sum(correct_values) / len(y_pred)
        return self.output

## Dense layer

In [3]:
class DenseLayer:

    def __init__(self,n_features, n_neurons):
        self.weights = torch.rand((n_features,n_neurons))
        self.bias = torch.rand(n_neurons)

    def forward(self,inputs):
        self.output = torch.matmul(inputs,self.weights) + self.bias

## Activation Simgoid

In [4]:
class Sigmoid_Activation:
    def forward(self,input):
        self.output = 1 / (1 + torch.exp(-input))
        return self.output

## Activation ReLU

In [5]:
class ReLU_Activation:
    def forward(self,inputs):
        self.output = torch.max(inputs,torch.tensor(0))
        return self.output



# Model

In [6]:
class Model:
    def __init__(self):
        self.layer1 = DenseLayer(2,2)
        self.layer1_activation = ReLU_Activation()
        self.output_layer = DenseLayer(2,2)
        self.output_layer_activation = Sigmoid_Activation()
        self.error = float('inf')

    def forward_prop(self,inputs):
        self.input = inputs
        self.layer1.forward(inputs)
        self.layer1_activation.forward(self.layer1.output)
        self.output_layer.forward(self.layer1_activation.output)
        self.output_layer_activation.forward(self.output_layer.output)

    def calc_error(self,y_true):
        if self.output_layer_activation.output.shape != y_true.shape:
            one_hot_notation = torch.zeros(self.output_layer_activation.output.shape)
            one_hot_notation[y_true] = 1
        else:
            one_hot_notation = y_true
        self.y_true = one_hot_notation
        self.error = (self.output_layer_activation.output - one_hot_notation)
        self.mse = torch.mean(self.error)

    def back_prop(self,lr):
        dloss_by_dsig = self.error
        dsig_by_layer2 = (torch.tensor([1]) - self.output_layer_activation.output) * self.output_layer_activation.output

        # layer 2
        back2 = dloss_by_dsig * dsig_by_layer2
        layer2_grad = torch.cat((self.layer1_activation.output.unsqueeze(dim=0),self.layer1_activation.output.unsqueeze(dim=0)),dim=0).T * back2
        self.output_layer.weights -= torch.tensor([lr]) * layer2_grad
        self.output_layer.bias -= torch.tensor([lr]) * back2

        # layer 1
        drelu_by_dlayer1 = self.layer1.output>0
        back1 = drelu_by_dlayer1 * torch.sum(self.output_layer.weights * back2,dim=1,keepdims=True).squeeze()
        layer1_grad = torch.cat((self.input.unsqueeze(dim=0),self.input.unsqueeze(dim=0)),dim=0).T * back1
        self.layer1.weights -= torch.tensor([lr]) * layer1_grad
        self.layer1.bias -= torch.tensor([lr]) * back1


    def fit(self,inputs,y_true,epoches=1000,lr=0.01):
        for epoch in range(epoches):
            for i in range(len(inputs)):
                self.forward_prop(inputs[i])
                self.calc_error(y_true[i])
                if self.mse < 0.1:
                    break
                self.back_prop(lr)
        error = torch.tensor([0,0],dtype = torch.float)
        for i in range(len(inputs)):
            self.forward_prop(inputs[i])
            self.calc_error(y_true[i])
            print("output:",self.output_layer_activation.output,"expected: ",self.y_true)
            print("squared error",self.error**2)
            print()
            error += self.error**2

        avg_error = error / len(inputs)
        print("average squared error",avg_error)
        print("over all mse:",torch.mean(avg_error))



In [10]:
model = Model()
inputs = torch.tensor([[1,2],[3,4],[5,6],[7,8],[9,1],[2,3]],dtype=torch.float)
y_true = torch.tensor([0,0,1,1,1,0])
model.fit(inputs,y_true,epoches=20000)

output: tensor([0.6600, 0.5358]) expected:  tensor([1., 0.])
squared error tensor([0.1156, 0.2871])

output: tensor([0.6623, 0.5359]) expected:  tensor([1., 0.])
squared error tensor([0.1140, 0.2872])

output: tensor([0.6647, 0.5359]) expected:  tensor([0., 1.])
squared error tensor([0.4418, 0.2154])

output: tensor([0.6671, 0.5360]) expected:  tensor([0., 1.])
squared error tensor([0.4450, 0.2153])

output: tensor([0.9710, 0.9148]) expected:  tensor([0., 1.])
squared error tensor([0.9429, 0.0073])

output: tensor([0.6612, 0.5359]) expected:  tensor([1., 0.])
squared error tensor([0.1148, 0.2871])

average squared error tensor([0.3624, 0.2166])
over all mse: tensor(0.2895)
